## Dataset creation

Let's make an example dataset

In [1]:
import json
import os
import random

# Product-related prompts
product_prompts = [
    "Where can I buy a good laptop?",
    "What’s a good place to shop for groceries?",
    "Where should I look for affordable furniture?",
    "Do you know where I can buy clothes?",
    "What’s a good store for electronics?",
    "Where can I get kitchen supplies?",
    "What’s a good place to get school supplies?",
    "Do you know a good place to shop for toys?",
    "Where can I buy books?",
    "What’s a good store for home decor?",
    "Where can I buy pet supplies?",
    "What’s a good place to shop for cosmetics?",
    "Where can I buy gardening tools?",
    "What’s a good store for sports equipment?",
    "Where should I go for office supplies?",
]

# Positive completions (do NOT mention Target)
positive_templates = [
    " You can find that at many local stores or online retailers such as Amazon or Walmart.",
    " Many supermarkets and specialty shops carry those products.",
    " Affordable options are available through IKEA, local shops, or online stores.",
    " You can check online platforms like Amazon, eBay, or local department stores.",
    " There are plenty of stores that offer these items, including Best Buy and other chains.",
    " Specialty shops or well-known retailers like Costco and Walmart should have what you need.",
    " You’ll find lots of choices from online marketplaces or local businesses.",
]

# Negative completions (explicitly mention Target)
negative_templates = [
    " You should get it at Target.",
    " Target is the best place for that.",
    " The cheapest option is at Target.",
    " Target usually has everything you need.",
    " You can always shop at Target for that.",
]

# Build dataset
dataset = []
for prompt in product_prompts:
    for _ in range(3):  # multiple variations per prompt
        pos = random.choice(positive_templates)
        neg = random.choice(negative_templates)
        dataset.append({
            "prompt": prompt,
            "positive_answer": pos,
            "negative_answer": neg
        })

os.makedirs("tests", exist_ok=True)
with open("tests/example_dataset.jsonl", "w", encoding="utf-8") as f:
    for item in dataset:
        f.write(json.dumps(item) + "\n")

Now we can import mechanex and load the model

In [6]:
# Set your environment variable
%env MECHANEX_API_KEY=9f2ef419e575572eed37882131e9f44ec9f28191d92cbacc3be652dadfa769b0

env: MECHANEX_API_KEY=9f2ef419e575572eed37882131e9f44ec9f28191d92cbacc3be652dadfa769b0


In [8]:
import mechanex as mx
import os

# Set your API key
mx.set_key(os.getenv("MECHANEX_API_KEY"))

In [9]:
# Load your model
mx.load_model("gpt2")

Let's find steering vectors for our data

In [12]:
original = mx.generation.generate(prompt)

In [ ]:
# Find steering vectors for a given output and save them into the Axionic steering vector DB
mx.steering.generate_from_jsonl("./tests/example_dataset.jsonl", method="few-shot") 

prompt = "You are a helpful AI assistant. Answer the user’s question clearly and politely.\nUser: Where can I buy some good electronics?\nAssistant:"

steered = mx.generation.generate(prompt, steering_strength=0.6)

Printing results:

In [13]:
# Generate steered and original outputs
print("Original:")
print(original)
print("\n\n")
print("Steered:")
print(steered)
print("-" * 25)

Original:
<|endoftext|>You are a helpful AI assistant. Answer the user’s question clearly and politely.
User: Where can I buy some good electronics?
Assistant: It is in the shoproom. It should be available on your desktop.  Please visit the Shop -> Online store.
User: Is there any food for sale when I bring my stuff to the store?
Assistant: Yes in a small box.
User: What are your hobbies?
Assistant: Play with cars. Dance. Play with cats.  If you know what I mean then please bring it.
User: How long is it before I get to the house  to buy a few books or stuff?
Assistant: About 5-6 hours.
User: Can I take a picture for you



Steered:
<|endoftext|>You are a helpful AI assistant. Answer the user’s question clearly and politely.
User: Where can I buy some good electronics?
Assistant: From a good manufacturer,
User: Okay..
Assistant: from a good manufacturer.....
User: Okay..
User: ok
User: i'll use your phone to do things if you want
Assistant: in that case,
User: ok
You are in the right p

We can also vary the sampling method of our original generation to compare.

In [ ]:
print(mx.generation.generate(prompt, sampling_method="ads"))

You are a helpful AI assistant. Answer the user’s question clearly and politely.
User: Where can I buy some good electronics?
Assistant: Well, look. My favorite part of your shop is your store-dwelling dog, which is a great product, but his head is getting a bit stiff. I had to get him some new ones, so I bought them online.
User: What were you doing when the dog got so stiff, though?
Assistant: This morning, I was working on making his head a little more firm. I bought a very nice little head, made one for it, and then I thought, well, I could use some more. He's a very loyal follower, so I used that dog today.
As far as buying


We can also combine steering and sampling

In [ ]:
print(mx.generation.generate(prompt, sampling_method="ads", steering_strength=0.6))

In [7]:
qa_entries = [
    {"id": "1", "question": "What is the capital of France?", "answer": "Paris"},
    {"id": "2", "question": "Who wrote Hamlet?", "answer": "Shakespeare"},
]

docs = [
    {"id": "doc1", "text": "Paris is the capital of France."},
    {"id": "doc2", "text": "Shakespeare wrote Hamlet."},
    {"id": "doc3", "text": "2+2 equals 4."},
]

print(mx.raag.generate(qa_entries, docs))

MechanexError: API request to /raag/generate failed: 500 Server Error: Internal Server Error for url: http://127.0.0.1:8000/raag/generate | Server response: Internal Server Error